# statsmodel
* hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığı
* **veriyi kullanarak bir OLS (Ordinary Least Squares) regresyon modeli oluşturur ve özniteliklerin anlamlılığını test eder.**

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#### California Housing veri setini yükleyelim

In [2]:
housing = fetch_california_housing()

In [3]:
X = pd.DataFrame(housing.data, columns=housing.feature_names)

In [4]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [5]:
y = pd.Series(housing.target, name='MedHouseVal')

In [6]:
y

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: MedHouseVal, Length: 20640, dtype: float64

#### Eğitim ve test verilerine bölelim

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Statsmodels ile OLS (Ordinary Least Squares) regresyonu

In [8]:
X_train_const = sm.add_constant(X_train)  # Sabit terim ekliyoruz

In [9]:
model = sm.OLS(y_train, X_train_const).fit()


In [10]:
# Model özetini yazdırma
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     3261.
Date:                Tue, 30 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:49:55   Log-Likelihood:                -17998.
No. Observations:               16512   AIC:                         3.601e+04
Df Residuals:                   16503   BIC:                         3.608e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -37.0233      0.728    -50.835      0.0

In [11]:
# p-değeri 0.05'ten küçük olan öznitelikleri seçiyoruz
significant_features = model.pvalues[model.pvalues < 0.05].index.tolist()
significant_features.remove('const')  # Sabit terimi listeden çıkarıyoruz

#### Anlamlı özniteliklerle yeni bir model oluşturuyoruz

In [12]:
X_train_sig = X_train[significant_features]
X_train_sig_const = sm.add_constant(X_train_sig)
model_sig = sm.OLS(y_train, X_train_sig_const).fit()

In [13]:
# Yeni model özetini yazdırma
print(model_sig.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     3727.
Date:                Tue, 30 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:55:42   Log-Likelihood:                -17998.
No. Observations:               16512   AIC:                         3.601e+04
Df Residuals:                   16504   BIC:                         3.607e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -37.0114      0.728    -50.865      0.0

## Yeni Modeli Test Etme ve Karşılaştırma

#### test verisi üzerinde tahminler

In [14]:
X_test_const = sm.add_constant(X_test)
y_pred = model.predict(X_test_const)

X_test_sig = X_test[significant_features]
X_test_sig_const = sm.add_constant(X_test_sig)
y_pred_sig = model_sig.predict(X_test_sig_const)

#### Performans karşılaştırması (Mean Squared Error)

In [15]:
mse_original = mean_squared_error(y_test, y_pred)
mse_significant = mean_squared_error(y_test, y_pred_sig)

print(f'Original Model MSE: {mse_original}')
print(f'Significant Features Model MSE: {mse_significant}')

Original Model MSE: 0.5558915986952462
Significant Features Model MSE: 0.5559531347340669


* Model performansını karşılaştırmak için ortalama kare hatası (MSE) kullanılır. MSE, modelin tahminlerinin gerçek değerlerden ne kadar sapma gösterdiğini ölçer; daha düşük bir MSE değeri, daha iyi bir model performansını gösterir.
* Her iki modelin de MSE değerleri birbirine çok yakın olduğundan, performans açısından büyük bir fark yoktur. Orijinal model biraz daha iyi performans gösteriyor olsa da, anlamlı özniteliklerle oluşturulan modelin avantajları göz önünde bulundurularak (örneğin, modelin daha basit ve yorumlanabilir olması) tercih edilebilir. MSE'nin çok yakın olması, anlamlı özniteliklerle oluşturulan modelin performans kaybı olmadan daha sade bir model sunduğunu gösterir.